In [1]:
import gym
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

2022-01-04 14:34:18.129399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-04 14:34:18.129492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Deterministic Maze

In [2]:
from FlagMaze_v0 import MazeEnv
env = MazeEnv(1000)

Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a X o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 


In [3]:
nb_actions = env.action_space.n

# Complex Neural Network for DQN, SARSA
CD_model = Sequential()
CD_model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
CD_model.add(Dense(16))
CD_model.add(Activation('relu'))
CD_model.add(Dense(16))
CD_model.add(Activation('relu'))
CD_model.add(Dense(16))
CD_model.add(Activation('relu'))
CD_model.add(Dense(nb_actions))
CD_model.add(Activation('linear'))

# Boltzmann Q Policy
BQ_policy = BoltzmannQPolicy()

# Sequential Memory
S_memory = SequentialMemory(limit=50000, window_length=1)

In [4]:
dqn = DQNAgent(model=CD_model, nb_actions=nb_actions, memory=S_memory, nb_steps_warmup=100, target_model_update=1e-2, policy=BQ_policy)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mse'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2022-01-04 14:34:57.947125: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 14:34:57.948702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-04 14:34:57.979056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-04 14:34:57.984062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-04 14:34:57.984693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 40:34 - reward: 0.0000e+00

/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 108s 11ms/step - reward: -0.2871
10 episodes - episode_reward: -287.100 [-563.000, -228.000] - loss: 0.202 - mse: 2.013 - mean_q: -0.949

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 108s 11ms/step - reward: -0.2369
10 episodes - episode_reward: -236.900 [-260.000, -201.000] - loss: 0.185 - mse: 3.142 - mean_q: -1.578

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 108s 11ms/step - reward: -0.2415
10 episodes - episode_reward: -241.500 [-281.000, -220.000] - loss: 0.173 - mse: 2.299 - mean_q: -1.234

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: -0.2401
10 episodes - episode_reward: -240.100 [-273.000, -211.000] - loss: 0.171 - mse: 1.362 - mean_q: -0.732

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 115s 12ms/step - reward: -0.2261
done, took 550.529 seconds


In [5]:
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: -979.000, steps: 1000
Episode 2: reward: -979.000, steps: 1000
Episode 3: reward: -979.000, steps: 1000
Episode 4: reward: -979.000, steps: 1000
Episode 5: reward: -979.000, steps: 1000
Episode 6: reward: -979.000, steps: 1000
Episode 7: reward: -979.000, steps: 1000
Episode 8: reward: -979.000, steps: 1000
Episode 9: reward: -979.000, steps: 1000
Episode 10: reward: -979.000, steps: 1000


In [6]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a X a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a X a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E


Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C 

...


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C

# Non-Deterministic Maze

In [7]:
from FlagMaze_v1 import MazeEnv
env1 = MazeEnv(1000)

Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a X o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 


In [8]:
# Complex Neural Network for DQN, SARSA
CD_model1 = Sequential()
CD_model1.add(Flatten(input_shape=(1,) + env1.observation_space.shape))
CD_model1.add(Dense(16))
CD_model1.add(Activation('relu'))
CD_model1.add(Dense(16))
CD_model1.add(Activation('relu'))
CD_model1.add(Dense(16))
CD_model1.add(Activation('relu'))
CD_model1.add(Dense(nb_actions))
CD_model1.add(Activation('linear'))

In [9]:
dqn1 = DQNAgent(model=CD_model1, nb_actions=nb_actions, memory=S_memory, nb_steps_warmup=100, target_model_update=1e-2, policy=BQ_policy)
dqn1.compile(Adam(learning_rate=1e-3), metrics=['mse'])
dqn1.fit(env1, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 123s 12ms/step - reward: -0.2922
10 episodes - episode_reward: -292.200 [-368.000, -236.000] - loss: 0.207 - mse: 4.643 - mean_q: -1.830

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 120s 12ms/step - reward: -0.2791
10 episodes - episode_reward: -279.100 [-307.000, -224.000] - loss: 0.228 - mse: 24.527 - mean_q: -5.393

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 122s 12ms/step - reward: -0.2869
10 episodes - episode_reward: -286.900 [-320.000, -260.000] - loss: 0.257 - mse: 31.557 - mean_q: -6.117

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 118s 12ms/step - reward: -0.2779
10 episodes - episode_reward: -277.900 [-300.000, -253.000] - loss: 0.274 - mse: 32.555 - mean_q: -6.178

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 108s 11ms/

In [10]:
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: -287.000, steps: 1000
Episode 2: reward: -273.000, steps: 1000
Episode 3: reward: -271.000, steps: 1000
Episode 4: reward: -287.000, steps: 1000
Episode 5: reward: -292.000, steps: 1000
Episode 6: reward: -249.000, steps: 1000
Episode 7: reward: -276.000, steps: 1000
Episode 8: reward: -287.000, steps: 1000
Episode 9: reward: -251.000, steps: 1000
Episode 10: reward: -263.000, steps: 1000


In [11]:
dqn1.test(env1, nb_episodes=1, visualize=True)

Testing for 1 episodes ...

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a X a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a X o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D * D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D X D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move right
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D X D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C 

A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D X D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D X D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D X D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D X D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C


Next action:Move right
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D X D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C 


Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D X D D C C C C C C C E E E E * 

Next action:Move up
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D X D D C C C C C C o o E E E E 
D D D D D D C C C C C


Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
X o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
X o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C


Next action:Move left
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D X D D C o C C C C C E E E E E 
D D D D D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C C C C E E E E * 

Next action:Move down
Map:
A A A A A * B B B B B B B E E E E E 
A A A A A A B B B B B B B E E E E E 
A A A A A A B * B B B B B E E E E E 
A A A A o A B B B B B B B E E * E E 
a a a a o a b b b B B B B E E E E E 
a a a a a o o b o o B B B E E E E E 
a o a a a a b b b B B B B E E E E E 
D o D D D D b b b B B B B E E E E E 
D D D D D D b o b B B B B E E E E E 
D D D D D D C o C C C C C E E E E E 
D D D X D D C C C C C * C E E E E E 
D G D D D D C C C C C C o o E E E E 
D D D D D D C C C C